In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import random

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
learning_rate = 0.000001
tf.set_random_seed(777)

batchsize = 50

n_InputUnit = 3648
n_1stHiddenUnit = 3648
n_2ndHiddenUnit = 500
n_3rdHiddenUnit = 200
n_4thHiddenUnit = 60
n_OutputUnit = 3

In [10]:
# Algorithm which decides standard for classifying data
# Algorithm which can detect some kind of different phenomena
W1 = np.random.normal(0.0,0.01,size = n_InputUnit)

batch_X = RealOESdata[Order*batchsize: (Order+1)*batchsize,:]
L = W1[0]*batch_X[0,0]
for i in range (batchsize):
    for j in range(3648):
        if i == 0 && j == 0:
            L = L
        else:
            L = np.hstack((L, W1[j]*batch_X[batch])))

W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [n_InputUnit, None]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None, n_OutputUnit])
keep_prob = tf.placeholder(tf.float32)

L2 = tf.nn.elu(tf.matmul(X, W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

L3 = tf.nn.elu(tf.matmul(L2_drop, W3) + b3)
L3_drop = tf.nn.dropout(L3, keep_prob)

L4 = tf.nn.elu(tf.matmul(L3_drop, W4) + b4)
logits = tf.matmul(L4, W5) + b5
Hypothesis = tf.nn.softmax(logits)

K = 0.0
for i in range(3648):
    K = K+abs(W1[i])

#tf.stop_gradient(Y)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits) + K)
#cost = cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(, Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)
prediction = tf.argmax(Hypothesis, 1)
correct_prediction = tf.equal(tf.argmax(Hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    
    
    for step in range (150001):
        Order = Sequence[step % 24]
        
        
        ### Y_ClassifiedResult_Power -> Power에 대한 one hot vector
        batch_Y = np.reshape(Y_ClassifiedResult_Press[Order*batchsize: (Order+1)*batchsize,:],
                             (batchsize, n_OutputUnit))
        session.run(train, feed_dict = {X: batch_X, Y: batch_Y, keep_prob: 0.5})
        MinibatchCost = session.run(cost, feed_dict = {X: batch_X, Y: batch_Y, keep_prob: 1.0})
        
        if step % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={X: batch_X, Y: batch_Y, keep_prob: 1.0})
            print ('Step %i: Minibatch Loss: %f, Minibatch Accuracy: %f' % (step, MinibatchCost,
                                                                            train_accuracy))
            
        if step % 24 == 0:
            random.shuffle(Sequence)
    
    RealOESdata_Reshaped = np.reshape(RealOESdata, (1200, 3648))
    
    ###Y_ClassifiedResult_Power
    train_accuracy = accuracy.eval(feed_dict = {X: RealOESdata_Reshaped, Y: Y_ClassifiedResult_Press, keep_prob: 1.0})        
    print('Full batch Accuracy: %f' % (train_accuracy))
    
    PredictedSimplifiedInputData = session.run(logits, feed_dict = {X : SimplifiedInput,keep_prob: 1.0})
    print(PredictedSimplifiedInputData)
    np.savetxt('Press_Weight.txt', PredictedSimplifiedInputData, delimiter='\t')
    
    PredictedSimplifiedInputDataLinear = session.run(WL4, feed_dict = {X : SimplifiedInput,keep_prob: 1.0})
    print(PredictedSimplifiedInputDataLinear)
    np.savetxt('Press_Weight2.txt', PredictedSimplifiedInputDataLinear, delimiter='\t')
    
    #for i in range(24):
    #    Result_batch_X = np.reshape(RealOESdata[i*batchsize: (i+1)*batchsize,:],
    #                                (batchsize, n_InputUnit))
    #    PredictedBatchData = session.run(prediction, feed_dict = {X : Result_batch_X, keep_prob: 1.0})
    #    
    #    ### Y_ClassifiedResult_Power
    #    Y_ClassifiedResult_Reshaped = np.reshape(Y_ClassifiedResult_Press[i*batchsize: (i+1)*batchsize,:],
    #                                             (batchsize, n_OutputUnit))
    #    
    #    if i % 1 == 0:
    #        print("Original Data")
    #        print(Y_ClassifiedResult_Reshaped[0,:])
    #    
    #    print(i)
    #    print("Result of predicting EntireData")
    #    print(PredictedBatchData)
    #        
    #print("####################################################")  
    #
    #for i in range(24):
    #    #
    #    Test_batch_X = np.reshape(TestOESdata[i*batchsize: (i+1)*batchsize,:],
    #                                (batchsize, n_InputUnit))
    #    
    #    
    #    PredictedTestBatchData = session.run(prediction, feed_dict = {X : Test_batch_X, keep_prob: 1.0})
    #    
    #    ###Y_Test_Power -> Test data의 power에 대한 one hot vector
    #    #
    #    Y_Test_Reshaped = np.reshape(Y_Test_Press[i*batchsize: (i+1)*batchsize,:],
    #                                             (batchsize, n_OutputUnit))
    #    
    #    if i % 1 == 0:
    #        print("Original Data")
    #        print(Y_Test_Reshaped[0,:])
    #    
    #    print(i)
    #    print("Result of Predicted Test Data")
    #    print(PredictedTestBatchData)
    ##
    #TestOESdata_Reshaped = np.reshape(TestOESdata, (1200,3648))
    #
    ####Y_Test_Power
    ##
    #train_accuracy = accuracy.eval(feed_dict = {X: TestOESdata_Reshaped, Y: Y_Test_Press, keep_prob: 1.0})        
    #print('Test Full batch Accuracy: %f' % (train_accuracy))          
    #